In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation

In [2]:
ret_decomp_dcf5 = \
          pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_PLS_1980_2019.csv', index_col=0).replace([np.inf, -np.inf], np.nan)
ret_decomp_dcf5['jdate'] = pd.to_datetime(ret_decomp_dcf5.jdate)

ret_decomp_dcf5_annual = ret_decomp_dcf5[ret_decomp_dcf5.jdate.dt.month==6]

regdf = ret_decomp_dcf5_annual[['jdate', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()


/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [44]:
regdf = ret_decomp_dcf5[['jdate', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()


In [54]:
def fama_macbeth_summary(p):
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['count', 'mean','std_error','tstat',]]

def ts_nw(xt):
    e = xt - np.mean(xt)
    T = e.shape[0]
    print(T)
    L = int(4*(T/100)**(2/9))
    w = 0
    for l in range(1, L+1):
        wl = 1-(l/(1+L))
        for t in range(l+1, T+1):
            w+= 2* wl * e[t-1] * e[t-l-1]
    print('w', w, 's', np.sum(e**2))
    S = w + np.sum(e**2)
    return np.sqrt(S)/T

def fama_macbeth_reg_panel_nw(regdf, xname='misp', yname='ret', 
                               csname='permno', tsname='jdate'): 
    # Panel factor setting FM reg
    gamma_cs = (regdf.groupby(tsname).apply(ols_coef, f'{yname} ~ {xname}'))
    gamma_cs = gamma_cs.rename(columns={"Intercept": "alpha_i_t", f"beta_{xname}": "lamba_t"})
    summary = fama_macbeth_summary(gamma_cs)
    nw_std_error_alpha = ts_nw(np.array(gamma_cs['alpha_i_t']))
    nw_std_error_beta = ts_nw(np.array(gamma_cs[xname]))
    summary['std_error_nw'] = np.array([nw_std_error_alpha, nw_std_error_beta])
    summary['tstat_nw'] = summary['mean'] / np.array([nw_std_error_alpha, nw_std_error_beta])
    return summary

In [55]:
fama_macbeth_reg_panel_nw(regdf, xname='me', yname='misp', 
                                 csname='permno', tsname='jdate')

480
w 2998.431467778709 s 617.69651939349
480
w 30.566622696237243 s 6.206065285073938


,count,mean,std_error,tstat,std_error_nw,tstat_nw
alpha_i_t,480.0,-9.433830,0.051832,-182.007308,0.125280,-75.302149
me,480.0,0.643487,0.005195,123.856743,0.012633,50.935198
